# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats

In [ ]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [ ]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

In [ ]:
data.head()

<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
</div>


Question 1) CLT applies because N is larger than 30. We can perform a 2 sample permutation test to determine how many outcomes result in a call-back deficit at least as extreme as the deficit observed in the dataframe data.
Question 2) The null hypothesis would be that companies call back black applicants as frequently as white applicants. The alternative hypothesis is that the rate of callbacks for black applicants is less than for white applicants.

In [26]:
w = data[data.race=='w']
b = data[data.race=='b']
print(np.mean(w.call)-np.mean(b.call))

0.0320328548551


In [53]:
# Your solution to Q3 here(margin of error, confidence interval, p-value using both frequentist and boostrapping methods )
#bootstrapping method
white = data[data.race=='w'].call.values
black = data[data.race=='b'].call.values
diff_of_means = np.mean(white) - np.mean(black) #calculate observed difference between callback rates

perm_replicates = np.empty(10000) #generate empty array
size = len(perm_replicates)
for i in range(size): #iterate over indices of range(size) to get a mean callback rate difference between races for each permutation)
    permutation_samples = np.random.permutation(np.concatenate((white,black)))
    white_sample = permutation_samples[:len(white)]
    black_sample = permutation_samples[len(white):]
    perm_replicates[i] = np.abs(np.mean(white_sample) - np.mean(black_sample))#get the difference in mean for each permutation of the concatenated array(100000 samples taken)

    
p_value = np.sum(perm_replicates > diff_of_means) / size
#print('p :', p_value)

confidence_interval = np.percentile(perm_replicates,[2.5,97.5])
#print(confidence_interval)



#frequentist tests
#compute confidence interval/margin of error: confidence interval = observed_diff +- margin of error 
#compute test statistic/z score: z score = (test statistic - hypothesized) / standard error
white_std = white.std()
black_std = black.std()

print('white std:\t',white_std,'\nblack std:\t',black_std,'\nmean-diff:\t',diff_of_means)
std_err = np.sqrt(white_std**2/len(white) + black_std**2/len(black))
print('error:',std_err)

confidence_interval_freq = [diff_of_means - 1.96 * std_err, diff_of_means + 1.96 * std_err]
print(confidence_interval_freq)

('white std:\t', 0.29528835, '\nblack std:\t', 0.24559963, '\nmean-diff:\t', 0.032032855)
('error:', 0.007783370603907358)
[0.016777448471402157, 0.047288261238719]


<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>

These tests tell the story that there is a significant difference in call back rate between white and black applicants even when resumes are equal/random for employers. This is clear because the p_value in the bootstrapping method was 0 (i.e. in 10000 permutations that created 10000 different "diff of mean" variables when the call back rates were assumed in equal, 0 produced a call back rate as extreme as the from the original data set. Also, the confidence interval at 95% did not include 0 which is another indicator that we can reject the null hypothesis at a 5% significance level. 

Based on this data alone, we cannot conclude that race was the driving factor in the difference of call back rates. Further investigation would be needed. At the very least, we can conclude that employers called back black applicants significantly less frequently than white applicants given that both had the same credentials. It would be beneficial to keep this in mind in our further investigation and carefully examine the correlation between race and any other factors that would also affect call back rate.

('p =', 0)
[0.         0.01724846]
